In [ ]:
import torch
print(torch.cuda.is_available())
!nvidia-smi

True
Wed Nov 12 16:33:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+

In [ ]:
import sys
!{sys.executable} -m pip install transformers datasets scikit-learn pandas numpy
print("Required libraries installed successfully.")

Required libraries installed successfully.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from datasets import DatasetDict, Dataset, load_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

print("All required modules imported successfully.")

All required modules imported successfully.


In [ ]:
import os

# Download the dataset
!wget http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

# Unzip the downloaded file
!unzip -o trainingandtestdata.zip

# Rename the extracted file to match the expected filename
# Check if the unzipped file exists before renaming
if os.path.exists('training.1600000.processed.noemoticon.csv'):
    os.rename('training.1600000.processed.noemoticon.csv', 'Twitter Sentiment140.csv')
    print("Dataset downloaded and renamed to 'Twitter Sentiment140.csv' successfully.")
else:
    print("Extracted file 'training.1600000.processed.noemoticon.csv' not found. Please check extraction.")

--2025-11-12 16:34:30--  http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip [following]
--2025-11-12 16:34:30--  https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81363704 (78M) [application/zip]
Saving to: ‘trainingandtestdata.zip’

trainingandtestdata 100%[===================>]  77.59M  43.6MB/s    in 1.8s    

2025-11-12 16:34:32 (43.6 MB/s) - ‘trainingandtestdata.zip’ saved [81363704/81363704]

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  
Dataset downloaded and renam

In [ ]:
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
df = pd.read_csv('Twitter Sentiment140.csv', encoding='ISO-8859-1', names=column_names)
print("Dataset loaded successfully. First 5 rows:")
print(df.head())

Dataset loaded successfully. First 5 rows:
   target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [ ]:
df = df.rename(columns={'target': 'sentiment'})
df = df[['text', 'sentiment']]
print("Columns renamed and selected. First 5 rows:")
print(df.head())

Columns renamed and selected. First 5 rows:
                                                text  sentiment
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...          0
1  is upset that he can't update his Facebook by ...          0
2  @Kenichan I dived many times for the ball. Man...          0
3    my whole body feels itchy and like its on fire           0
4  @nationwideclass no, it's not behaving at all....          0


In [ ]:
df['sentiment'] = df['sentiment'].replace({0: 0, 2: 1, 4: 2})
print("Sentiment labels mapped. Value counts:")
print(df['sentiment'].value_counts())
print("First 5 rows with mapped sentiment:")
print(df.head())

Sentiment labels mapped. Value counts:
sentiment
0    800000
2    800000
Name: count, dtype: int64
First 5 rows with mapped sentiment:
                                                text  sentiment
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...          0
1  is upset that he can't update his Facebook by ...          0
2  @Kenichan I dived many times for the ball. Man...          0
3    my whole body feels itchy and like its on fire           0
4  @nationwideclass no, it's not behaving at all....          0


In [ ]:
df_sampled = df.sample(n=10000, random_state=42)
print("Sampled DataFrame shape:", df_sampled.shape)
print("Sampled DataFrame head:")
print(df_sampled.head())

Sampled DataFrame shape: (10000, 2)
Sampled DataFrame head:
                                                     text  sentiment
541200             @chrishasboobs AHHH I HOPE YOUR OK!!!           0
750     @misstoriblack cool , i have no tweet apps  fo...          0
766711  @TiannaChaos i know  just family drama. its la...          0
285055  School email won't open  and I have geography ...          0
705995                             upper airways problem           0


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42, stratify=df_sampled['sentiment'])

print("Training DataFrame shape:", train_df.shape)
print("Testing DataFrame shape:", test_df.shape)
print("Training DataFrame head:")
print(train_df.head())
print("Testing DataFrame head:")
print(test_df.head())

Training DataFrame shape: (8000, 2)
Testing DataFrame shape: (2000, 2)
Training DataFrame head:
                                                      text  sentiment
1522078  @mtrh Thank you  If I manage to get through to...          2
656580   does ma hair look ok for prom??? D= i hate no ...          0
1201115  @JACKtheTRIPPER ur not following me me, thus i...          2
637495                    @chaaachaaa That's really mean.           0
869410   @SpeaksBeliefs LOL eye crusties, not cute! But...          2
Testing DataFrame head:
                                                      text  sentiment
442456   Lalala....bored at work and wondering why nobo...          0
1409168  has started to see an improvement with her mus...          2
734597   Mmm hot tub at a friends place... felt great! ...          0
971928   @SUNSHiNE_215 Thank you girl! I told you twitt...          2
750855   Abou to go et my rn on at Kenneth Hahn..  I so...          0


In [ ]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

# Create a DatasetDict
hf_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print("Hugging Face DatasetDict created successfully.")
print("Train dataset features:", hf_dataset['train'].features)
print("Test dataset features:", hf_dataset['test'].features)
print("First 5 examples from training dataset:")
print(hf_dataset['train'][:5])

Hugging Face DatasetDict created successfully.
Train dataset features: {'text': Value('string'), 'sentiment': Value('int64')}
Test dataset features: {'text': Value('string'), 'sentiment': Value('int64')}
First 5 examples from training dataset:
{'text': ["@mtrh Thank you  If I manage to get through today, I'll let you know.", 'does ma hair look ok for prom??? D= i hate no hats  http://tinyurl.com/kwksv7', '@JACKtheTRIPPER ur not following me me, thus i cannot DM you... but its very easy. Someone very close to you has it and is happy to share ', "@chaaachaaa That's really mean. ", "@SpeaksBeliefs LOL eye crusties, not cute! But your actions void all non cuteness right now! You go daddy! That's tooooo sweet! "], 'sentiment': [2, 0, 2, 0, 2]}


In [ ]:
model_name = 'distilbert-base-uncased' # You can switch to 'roberta-base' or 'bert-base-multilingual-cased'

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Tokenizer for {model_name} initialized.")

# Create a tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to both train and test datasets
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)
print("Datasets tokenized.")

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
print(f"Model {model_name} initialized with {model.num_labels} output labels.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Datasets tokenized.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model distilbert-base-uncased initialized with 3 output labels.


In [ ]:
model_name = 'distilbert-base-uncased' # You can switch to 'roberta-base' or 'bert-base-multilingual-cased'

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"Tokenizer for {model_name} initialized.")

# Create a tokenize function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to both train and test datasets
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True)
print("Datasets tokenized.")

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
print(f"Model {model_name} initialized with {model.num_labels} output labels.")

Tokenizer for distilbert-base-uncased initialized.


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Datasets tokenized.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model distilbert-base-uncased initialized with 3 output labels.


In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
tokenized_datasets.set_format("torch")

print("Dataset format set to PyTorch and 'sentiment' column renamed to 'labels'.")
print("First example from training dataset with new format and renamed label column:")
print(tokenized_datasets['train'][0])

Dataset format set to PyTorch and 'sentiment' column renamed to 'labels'.
First example from training dataset with new format and renamed label column:
{'text': "@mtrh Thank you  If I manage to get through today, I'll let you know.", 'labels': tensor(2), 'input_ids': tensor([  101,  1030, 11047, 25032,  4067,  2017,  2065,  1045,  6133,  2000,
         2131,  2083,  2651,  1010,  1045,  1005,  2222,  2292,  2017,  2113,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,  

In [ ]:
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments, Trainer

# 1. Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    fp16=torch.cuda.is_available(), # Enable mixed-precision training if CUDA is available
    learning_rate=5e-5
)

# 2. Define compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# 3. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

print("Trainer initialized. Starting training...")

# 4. Start training the model
trainer.train()

print("Training complete. Saving model...")

# 5. Save the trained model
save_directory = "/content/sentiment_model"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Trainer initialized. Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nayankshitij128 (nayankshitij128-bennett-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,0.943600
200,0.611000
300,0.538100
400,0.512700
500,0.527900
600,0.506700
700,0.479600
800,0.496900
900,0.441100
1000,0.456600


Training complete. Saving model...
Model and tokenizer saved to /content/sentiment_model


In [ ]:
import os

# Verify the presence of SemEval.csv (Instruction 1)
file_path = 'SemEval.csv'
if not os.path.exists(file_path):
    print(f"Error: {file_path} not found. Please ensure the file is present in the current directory.")
else:
    print(f"{file_path} found. Proceeding to load.")

    # Load the SemEval.csv file into a pandas DataFrame named semeval_df (Instruction 2)
    semeval_df = pd.read_csv(file_path, encoding='utf-8', comment='#')
    print("SemEval dataset loaded successfully. First 5 rows:")
    print(semeval_df.head())


SemEval.csv found. Proceeding to load.
SemEval dataset loaded successfully. First 5 rows:
     id                                           Sentence     Aspect Term  \
0  2339  I charge it at night and skip taking the cord ...            cord   
1  2339  I charge it at night and skip taking the cord ...    battery life   
2  1316  The tech guy then said the service center does...  service center   
3  1316  The tech guy then said the service center does...    "sales" team   
4  1316  The tech guy then said the service center does...        tech guy   

   polarity   from     to  
0   neutral   41.0   45.0  
1  positive   74.0   86.0  
2  negative   27.0   41.0  
3  negative  109.0  121.0  
4   neutral    4.0   12.0  


In [ ]:
semeval_df = semeval_df.rename(columns={'Sentence': 'text'})
print("Column 'Sentence' renamed to 'text'. First 5 rows:")
print(semeval_df.head())

Column 'Sentence' renamed to 'text'. First 5 rows:
     id                                               text     Aspect Term  \
0  2339  I charge it at night and skip taking the cord ...            cord   
1  2339  I charge it at night and skip taking the cord ...    battery life   
2  1316  The tech guy then said the service center does...  service center   
3  1316  The tech guy then said the service center does...    "sales" team   
4  1316  The tech guy then said the service center does...        tech guy   

   polarity   from     to  
0   neutral   41.0   45.0  
1  positive   74.0   86.0  
2  negative   27.0   41.0  
3  negative  109.0  121.0  
4   neutral    4.0   12.0  


In [ ]:
label_column_names = ['Label', 'Sentiment', 'target', 'label', 'polarity']
identified_label_column = None

for col_name in label_column_names:
    if col_name in semeval_df.columns:
        identified_label_column = col_name
        break

if identified_label_column:
    print(f"Identified label column: '{identified_label_column}'")
else:
    print("No common label column found. Please manually specify.")
    # Fallback if no common column is found, assuming 'polarity' based on prior output
    identified_label_column = 'polarity' # Manual specification as a fallback

# Displaying value counts of the identified label column to confirm content
print("Value counts of the identified label column:")
print(semeval_df[identified_label_column].value_counts())

Identified label column: 'polarity'
Value counts of the identified label column:
polarity
positive    3248
negative    1704
neutral     1107
conflict     136
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the identified label column
semeval_df['labels'] = label_encoder.fit_transform(semeval_df[identified_label_column])

print("Original label column value counts:")
print(semeval_df[identified_label_column].value_counts())
print("\nNumerical labels value counts:")
print(semeval_df['labels'].value_counts())
print("\nMapping of original labels to numerical codes:")
for i, label in enumerate(label_encoder.classes_):
    print(f"{label}: {i}")

print("\nFirst 5 rows with new numerical 'labels' column:")
print(semeval_df.head())

Original label column value counts:
polarity
positive    3248
negative    1704
neutral     1107
conflict     136
Name: count, dtype: int64

Numerical labels value counts:
labels
3    3248
4    3201
1    1704
2    1107
0     136
Name: count, dtype: int64

Mapping of original labels to numerical codes:
conflict: 0
negative: 1
neutral: 2
positive: 3
nan: 4

First 5 rows with new numerical 'labels' column:
     id                                               text     Aspect Term  \
0  2339  I charge it at night and skip taking the cord ...            cord   
1  2339  I charge it at night and skip taking the cord ...    battery life   
2  1316  The tech guy then said the service center does...  service center   
3  1316  The tech guy then said the service center does...    "sales" team   
4  1316  The tech guy then said the service center does...        tech guy   

   polarity   from     to  labels  
0   neutral   41.0   45.0       2  
1  positive   74.0   86.0       3  
2  negative   27.

In [ ]:
from datasets import Dataset

semeval_dataset = Dataset.from_pandas(semeval_df, preserve_index=False)

print("SemEval DataFrame converted to Hugging Face Dataset successfully.")
print("SemEval Dataset features:", semeval_dataset.features)
print("First 5 examples from SemEval Dataset:")
print(semeval_dataset[:5])

SemEval DataFrame converted to Hugging Face Dataset successfully.
SemEval Dataset features: {'id': Value('string'), 'text': Value('string'), 'Aspect Term': Value('string'), 'polarity': Value('string'), 'from': Value('float64'), 'to': Value('float64'), 'labels': Value('int64')}
First 5 examples from SemEval Dataset:
{'id': ['2339', '2339', '1316', '1316', '1316'], 'text': ['I charge it at night and skip taking the cord with me because of the good battery life.', 'I charge it at night and skip taking the cord with me because of the good battery life.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.', 'The tech guy then said the service center does not do 1-to-1 exchange and I 

In [ ]:
filtered_semeval_dataset = semeval_dataset.filter(lambda example: example['text'] is not None and isinstance(example['text'], str))

tokenized_semeval_dataset = filtered_semeval_dataset.map(tokenize_function, batched=True)
print("SemEval dataset tokenized successfully.")
print("First 5 examples from tokenized SemEval Dataset:")
print(tokenized_semeval_dataset[:5])

Filter:   0%|          | 0/9396 [00:00<?, ? examples/s]

Map:   0%|          | 0/7796 [00:00<?, ? examples/s]

SemEval dataset tokenized successfully.
First 5 examples from tokenized SemEval Dataset:
{'id': ['2339', '2339', '1316', '1316', '1316'], 'text': ['I charge it at night and skip taking the cord with me because of the good battery life.', 'I charge it at night and skip taking the cord with me because of the good battery life.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.'], 'Aspect Term': ['cord', 'battery life', 'service center', '"sales" team', 'tech guy'], 'polarity': ['neutral', 'positive'

In [ ]:
tokenized_semeval_dataset.set_format("torch")

print("SemEval Dataset: 'labels' column confirmed and format set to PyTorch.")
print("SemEval Dataset features:", tokenized_semeval_dataset.features)
print("First 5 examples from tokenized SemEval Dataset with PyTorch format:")
print(tokenized_semeval_dataset[:5])

SemEval Dataset: 'labels' column confirmed and format set to PyTorch.
SemEval Dataset features: {'id': Value('string'), 'text': Value('string'), 'Aspect Term': Value('string'), 'polarity': Value('string'), 'from': Value('float64'), 'to': Value('float64'), 'labels': Value('int64'), 'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8'))}
First 5 examples from tokenized SemEval Dataset with PyTorch format:
{'id': ['2339', '2339', '1316', '1316', '1316'], 'text': ['I charge it at night and skip taking the cord with me because of the good battery life.', 'I charge it at night and skip taking the cord with me because of the good battery life.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.', 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop whi

In [ ]:
num_semeval_labels = len(tokenized_semeval_dataset.unique('labels'))
print(f"Number of unique labels in SemEval dataset: {num_semeval_labels}")

# 1. Load the previously saved model and tokenizer
loaded_tokenizer = AutoTokenizer.from_pretrained("/content/sentiment_model")
loaded_model = AutoModelForSequenceClassification.from_pretrained(
    "/content/sentiment_model",
    num_labels=num_semeval_labels,
    ignore_mismatched_sizes=True
)

print("Previously saved model and tokenizer loaded successfully.")
print(f"Loaded model configured with {loaded_model.num_labels} output labels.")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/sentiment_model and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of unique labels in SemEval dataset: 5
Previously saved model and tokenizer loaded successfully.
Loaded model configured with 5 output labels.


In [ ]:
from sklearn.metrics import accuracy_score

# 2. Define compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# 3. Define TrainingArguments for fine-tuning
training_args_finetune = TrainingArguments(
    output_dir="./semeval_results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    fp16=torch.cuda.is_available(), # Enable mixed-precision training if CUDA is available
    logging_dir="./semeval_logs",
    logging_steps=100,
)

print("TrainingArguments for fine-tuning defined.")

# 4. Initialize Trainer instance
trainer_finetune = Trainer(
    model=loaded_model,
    args=training_args_finetune,
    train_dataset=tokenized_semeval_dataset,
    eval_dataset=tokenized_semeval_dataset, # Evaluate on the same dataset for this task
    compute_metrics=compute_metrics,
)

print("Trainer for fine-tuning initialized. Starting fine-tuning...")

# 5. Start fine-tuning the model
trainer_finetune.train()

print("Fine-tuning complete. Saving fine-tuned model...")

# 6. Save the fine-tuned model and tokenizer
final_save_directory = "/content/final_model"
trainer_finetune.save_model(final_save_directory)
loaded_tokenizer.save_pretrained(final_save_directory)

print(f"Fine-tuned model and tokenizer saved to {final_save_directory}")

TrainingArguments for fine-tuning defined.
Trainer for fine-tuning initialized. Starting fine-tuning...


Step,Training Loss
100,1.252300
200,1.057900
300,0.997900
400,0.919600
500,0.863700
600,0.813700
700,0.796000
800,0.806100
900,0.781500
1000,0.713700


Step,Training Loss
100,1.252300
200,1.057900
300,0.997900
400,0.919600
500,0.863700
600,0.813700
700,0.796000
800,0.806100
900,0.781500
1000,0.713700


Fine-tuning complete. Saving fine-tuned model...
Fine-tuned model and tokenizer saved to /content/final_model


In [ ]:
from transformers import pipeline

# 1. Initialize a text classification pipeline
sentiment_pipeline = pipeline(
    task='sentiment-analysis',
    model='/content/final_model',
    tokenizer='/content/final_model'
)
print("Text classification pipeline initialized.")

# 2. Create a list of example social media posts
example_posts = [
    "This is absolutely wonderful! I'm so excited!", # Positive emotion
    "I'm so happy I could scream! (totally not sarcasm)", # Sarcasm with positive wording
    "Just what I needed, a flat tire. My day is complete.", # Sarcasm with negative event
    "Feeling incredibly sad after watching that movie.", # Negative emotion
    "My cat just did the cutest thing, I can't even!", # Positive sentiment
    "Another glorious Monday, can't wait for the week.", # Sarcasm
    "This is fine. Everything is absolutely fine. (internal screaming)" # Sarcasm/Hidden emotion
]

print("Example posts created.")

# 3. Pass the list of example posts to the pipeline
predictions = sentiment_pipeline(example_posts)

print("Predictions generated. Displaying results:")

# 4. Iterate through the results and print each example post with its prediction
for i, (post, prediction) in enumerate(zip(example_posts, predictions)):
    print(f"\n--- Example Post {i+1} ---")
    print(f"Post: {post}")
    print(f"Prediction: Label '{prediction['label']}' with score {prediction['score']:.4f}")

Device set to use cuda:0


Text classification pipeline initialized.
Example posts created.
Predictions generated. Displaying results:

--- Example Post 1 ---
Post: This is absolutely wonderful! I'm so excited!
Prediction: Label 'LABEL_4' with score 0.9744

--- Example Post 2 ---
Post: I'm so happy I could scream! (totally not sarcasm)
Prediction: Label 'LABEL_4' with score 0.9853

--- Example Post 3 ---
Post: Just what I needed, a flat tire. My day is complete.
Prediction: Label 'LABEL_4' with score 0.9682

--- Example Post 4 ---
Post: Feeling incredibly sad after watching that movie.
Prediction: Label 'LABEL_4' with score 0.6391

--- Example Post 5 ---
Post: My cat just did the cutest thing, I can't even!
Prediction: Label 'LABEL_4' with score 0.9619

--- Example Post 6 ---
Post: Another glorious Monday, can't wait for the week.
Prediction: Label 'LABEL_3' with score 0.5417

--- Example Post 7 ---
Post: This is fine. Everything is absolutely fine. (internal screaming)
Prediction: Label 'LABEL_4' with score 0.9

In [ ]:
# 1. Split SemEval dataset into train and test sets (80/20 ratio)
semeval_train_ds, semeval_test_ds = filtered_semeval_dataset.train_test_split(test_size=0.2, seed=42).values()

print(f"SemEval Training dataset size: {len(semeval_train_ds)}")
print(f"SemEval Test dataset size: {len(semeval_test_ds)}")

# 2. Tokenize new splits (already tokenized in a previous step, but re-applying to ensure consistency)
tokenized_semeval_train_ds = semeval_train_ds.map(tokenize_function, batched=True)
tokenized_semeval_test_ds = semeval_test_ds.map(tokenize_function, batched=True)

# Set format to PyTorch
tokenized_semeval_train_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_semeval_test_ds.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print("SemEval datasets split, tokenized, and formatted for PyTorch.")

# 3. Initialize a new Trainer for evaluation
# Use the loaded_model (which is the fine-tuned model /content/final_model)

training_args_eval = TrainingArguments(
    output_dir="./semeval_eval_results",
    per_device_eval_batch_size=8,
    fp16=torch.cuda.is_available(),
    report_to="none" # Disable logging to wandb for evaluation only
)

trainer_eval = Trainer(
    model=loaded_model, # Use the fine-tuned model
    args=training_args_eval,
    eval_dataset=tokenized_semeval_test_ds,
    compute_metrics=compute_metrics,
    tokenizer=loaded_tokenizer
)

print("Trainer initialized for SemEval evaluation. Starting evaluation...")

# 4. Evaluate the model
evaluation_results = trainer_eval.evaluate()

# 5. Display results
print("\nEvaluation Results on SemEval Test Set:")
for metric_name, value in evaluation_results.items():
    print(f"{metric_name}: {value:.4f}")

SemEval Training dataset size: 6236
SemEval Test dataset size: 1560


Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

Map:   0%|          | 0/1560 [00:00<?, ? examples/s]

SemEval datasets split, tokenized, and formatted for PyTorch.
Trainer initialized for SemEval evaluation. Starting evaluation...


/tmp/ipython-input-2474092443.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_eval = Trainer(



Evaluation Results on SemEval Test Set:
eval_loss: 0.4658
eval_model_preparation_time: 0.0015
eval_accuracy: 0.8385
eval_runtime: 1.9782
eval_samples_per_second: 788.5960
eval_steps_per_second: 98.5750


In [ ]:
#The subtask is complete. All instructions have been successfully executed.

In [ ]:
# ==========================================================
# 🔥 Sentiment + Emotion / Sarcasm Detection Pipeline
# ==========================================================

# 1️⃣ Install dependencies
!pip install -q transformers datasets torch scikit-learn pandas numpy

# 2️⃣ Import libraries
import pandas as pd, numpy as np, torch, os
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, pipeline
)

# ==========================================================
# 3️⃣ Load and trim Sentiment140 (base dataset)
# ==========================================================
sent_path = "/content/Twitter Sentiment140.csv"  # Your uploaded file name

df = pd.read_csv(sent_path, encoding='latin-1')
df.columns = ['target','ids','date','flag','user','text']
df = df[['text','target']]
df['target'] = df['target'].map({0:0, 2:1, 4:2})  # 0=neg,1=neu,2=pos

# Keep 20k samples for quick training (changed from 10k)
df = df.sample(20000, random_state=42).reset_index(drop=True)
print(f"✅ Loaded {len(df)} tweets from Sentiment140")

# ==========================================================
# 4️⃣ Split into train/test
# ==========================================================
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['target'], test_size=0.2, random_state=42
)
train_df = pd.DataFrame({'text':train_texts,'label':train_labels})
test_df  = pd.DataFrame({'text':test_texts,'label':test_labels})
train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

# ==========================================================
# 5️⃣ Base Model: DistilBERT (changeable)
# ==========================================================
model_name = "distilbert-base-uncased"   # Change to "roberta-base" or "bert-base-multilingual-cased" if you want
tokenizer  = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)
train_ds.set_format('torch', columns=['input_ids','attention_mask','label'])
test_ds.set_format('torch', columns=['input_ids','attention_mask','label'])

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# ==========================================================
# 6️⃣ Train Sentiment140 model
# ==========================================================
training_args = TrainingArguments(
    output_dir="./results_sent140",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    learning_rate=5e-5,
    eval_strategy="epoch",
    fp16=torch.cuda.is_available(),
    logging_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)
trainer.train()
trainer.save_model("/content/sentiment_model")
print("✅ Base Sentiment140 model trained & saved!")

# ==========================================================
# 7️⃣ Load SemEval (merged) dataset for fine-tuning
#    (This section now re-uses the already preprocessed semeval_dataset and related variables)
# ==========================================================
semeval_path = "/content/SemEval.csv"   # Your merged SemEval file

if os.path.exists(semeval_path):
    # Re-use the already preprocessed and tokenized SemEval dataset from previous steps
    # This assumes 'tokenized_semeval_dataset' and 'num_semeval_labels' are available in the global scope
    # If you run this cell independently, ensure these variables are defined.

    # Get the number of labels for the fine-tuning model from the preprocessed data
    # num_semeval_labels was calculated in a previous step from tokenized_semeval_dataset.unique('labels')
    # For this consolidated block, we'll ensure it's calculated here if run independently
    # If the notebook is run top-to-bottom, num_semeval_labels is already defined.

    if 'tokenized_semeval_dataset' not in globals() or 'num_semeval_labels' not in globals():
        print("⚠️ Re-running SemEval preprocessing due to missing global variables.")
        # Original semeval_df loading and preprocessing steps needed if running this cell alone
        # This part is a fallback to ensure the code works if executed out of order
        semeval_df_fallback = pd.read_csv(semeval_path, encoding='utf-8', comment='#')
        semeval_df_fallback = semeval_df_fallback.rename(columns={'Sentence': 'text'})
        from sklearn.preprocessing import LabelEncoder
        label_encoder_fallback = LabelEncoder()
        semeval_df_fallback['labels'] = label_encoder_fallback.fit_transform(semeval_df_fallback['polarity'])
        filtered_semeval_dataset = Dataset.from_pandas(semeval_df_fallback, preserve_index=False)
        filtered_semeval_dataset = filtered_semeval_dataset.filter(lambda example: example['text'] is not None and isinstance(example['text'], str))
        tokenized_semeval_dataset = filtered_semeval_dataset.map(tokenize, batched=True)
        tokenized_semeval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
        num_semeval_labels = len(pd.Series(tokenized_semeval_dataset['labels']).unique())


    print(f"✅ Loaded SemEval dataset with {len(tokenized_semeval_dataset)} samples for fine-tuning.")

    # Load base model and fine-tune
    loaded_model_for_finetuning = AutoModelForSequenceClassification.from_pretrained(
        "/content/sentiment_model",
        num_labels=num_semeval_labels,
        ignore_mismatched_sizes=True # Add this argument to handle the size mismatch
    )

    training_args2 = TrainingArguments(
        output_dir="./results_semeval",
        num_train_epochs=2,
        per_device_train_batch_size=8,
        learning_rate=2e-5,
        eval_strategy="epoch",
        fp16=torch.cuda.is_available(),
        logging_strategy="epoch"
    )

    trainer2 = Trainer(
        model=loaded_model_for_finetuning,
        args=training_args2,
        train_dataset=tokenized_semeval_dataset,
        eval_dataset=tokenized_semeval_dataset, # Evaluate on the same dataset for this task, will be addressed with proper test split below
        tokenizer=tokenizer
    )
    trainer2.train()
    trainer2.save_model("/content/final_model")
    final_model_path = "/content/final_model"
else:
    print("⚠️ SemEval.csv not found, skipping fine-tuning.")
    final_model_path = "/content/sentiment_model"

# ==========================================================
# 8️⃣ Prediction demo
# ==========================================================
clf = pipeline("text-classification", model=final_model_path, tokenizer=tokenizer)

examples = [
    "I love how chill today feels 😌",
    "Wow, another network outage, amazing 🙄",
    "Aaj ka mood full mast hai bro 🔥",
    "This movie was a total waste of time."
]
print("\n🔍 Example Predictions:")
for t in examples:
    print(f"{t} → {clf(t)}")

print("\n✅ Project Complete — Sentiment + Emotion model ready.")

✅ Loaded 20000 tweets from Sentiment140


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2886568513.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.469600,0.438728


...

✅ Base Sentiment140 model trained & saved!
✅ Loaded SemEval dataset with 7796 samples for fine-tuning.


/tmp/ipython-input-2886568513.py:133: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer2 = Trainer(


Epoch,Training Loss,Validation Loss
1,0.916900,0.621887
2,0.585800,0.455983


Device set to use cuda:0



🔍 Example Predictions:
I love how chill today feels 😌 → [{'label': 'LABEL_4', 'score': 0.6183791160583496}]
Wow, another network outage, amazing 🙄 → [{'label': 'LABEL_4', 'score': 0.7963340282440186}]
Aaj ka mood full mast hai bro 🔥 → [{'label': 'LABEL_3', 'score': 0.7028079628944397}]
This movie was a total waste of time. → [{'label': 'LABEL_1', 'score': 0.7483070492744446}]

✅ Project Complete — Sentiment + Emotion model ready.
